In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05151009559631348
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 11.030434721101374
200000 22.088775840350497
300000 27.211234187958148
400000 25.764664561559464
500000 26.959919063954104
600000 27.950919044893663
700000 34.3999012413922
800000 36.97675520579547
900000 42.98262192383728
1000000 46.56014709208052
1100000 47.37248303646454
1200000 34.40139767975274
1300000 24.89635049657749
1400000 45.19812217327625
1500000 45.89204161918631
1600000 46.16475934669116
1700000 46.60479353358528
1800000 49.176520195151305
1900000 48.209354470360104
2000000 46.70843807351676
2100000 46.39075814022716
2200000 46.42662972674978
2300000 46.78205651831177
2400000 38.399103953637166
2500000 19.29181338076601
2600000 44.58946775799275
2700000 44.451337091091425
2800000 43.88512082668096
2900000 43.68654610884256
3000000 44.17187515

29700000 48.862494565838695
29800000 47.4383616677779
29900000 47.05039239382584
30000000 45.038304576020614
30100000 23.14068242416849
30200000 37.05034555628681
30300000 45.33010480940713
30400000 46.37865066711849
30500000 46.98490796710398
30600000 46.52566261763188
30700000 47.54195720241443
30800000 48.37268504524608
30900000 48.88342025096239
31000000 48.25706978631246
31100000 23.525539322263725
31200000 45.88966094101138
31300000 49.205047572408084
31400000 48.123503278950565
31500000 41.667572514269736
31600000 26.040424268819546
31700000 47.39293396470179
31800000 48.13442930412966
31900000 48.4303016734749
32000000 46.43931165524759
32100000 48.519003812460305
32200000 46.193386123480444
32300000 45.69027006245087
32400000 43.71583560802541
32500000 25.102840610411647
32600000 34.185263980086276
32700000 43.312120263690076
32800000 45.996206970649034
32900000 46.67719961532361
33000000 46.680875597081545
33100000 46.28707385035742
33200000 48.58154879205657
33300000 48.7191

59800000 22.612801383810705
59900000 46.436553809409624
60000000 45.612332638730464
60100000 45.62228595402933
60200000 46.22400978314532
60300000 45.673818756497745
60400000 45.1107427264092
60500000 46.210186591686394
60600000 35.95925874333048
60700000 27.253288608898874
60800000 43.196777144613236
60900000 42.47282172253282
61000000 42.94415738755441
61100000 41.22136450551831
61200000 40.62323006809818
61300000 39.146158779944855
61400000 13.720829511194726
61500000 40.43477811505118
61600000 39.77034400421137
61700000 41.52640584248516
61800000 39.896865086584526
61900000 42.411415447722746
62000000 42.5882254111732
62100000 36.34006561662884
62200000 28.640017575878765
62300000 46.41150693556378
62400000 45.78607293836057
62500000 44.17438431412154
62600000 45.522468345478494
62700000 46.95780370340179
62800000 46.0153155318599
62900000 47.20572049994911
63000000 44.48463776051657
63100000 19.004578674662575
63200000 44.587875358175005
63300000 44.30044873300448
63400000 42.3303

89900000 36.917981110989906
90000000 39.27897215121015
90100000 37.01995583359977
90200000 37.5399543122814
90300000 12.789430340852132
90400000 38.63698473058822
90500000 38.51306609026656
90600000 36.99530097170481
90700000 37.531890616306285
90800000 20.36737362234564
90900000 35.179214308792616
91000000 40.491157339970705
91100000 39.437847639469524
91200000 42.687126011478284
91300000 41.798328548506845
91400000 40.85915609616145
91500000 42.1528307597002
91600000 43.74333613680545
91700000 42.567814908693514
91800000 43.50496163101584
91900000 41.96711597028556
92000000 42.5605594970001
92100000 34.25214092679862
92200000 20.54006884056848
92300000 39.66762212443021
92400000 37.604823863768445
92500000 40.11732080883177
92600000 39.880463141319225
92700000 9.559772526176197
92800000 41.46013365951792
92900000 38.308265794252826
93000000 39.7026574468676
93100000 38.3867400366755
93200000 17.701491955180263
93300000 39.25074765309672
93400000 42.36888070669825
93500000 41.92625407

119300000 34.57512223581859
119400000 34.02516700613779
119500000 18.313843368646943
119600000 35.183382326105416
119700000 35.9015402321178
119800000 34.37761562551577
119900000 37.47914193391963
120000000 38.69124314839565
120100000 38.84007726087796
120200000 40.25439629419912
120300000 39.7388426993043
120400000 39.70740509840077
120500000 41.18692102091376
120600000 39.76676098622454
120700000 39.06695017702267
120800000 37.38208930932558
120900000 37.95353337251367
121000000 36.12313954258591
121100000 37.334856033774244
121200000 28.138306520156707
121300000 21.994456257491805
121400000 35.0483361553506
121500000 27.96145225954658
121600000 19.45775663001943
121700000 34.83198796206347
121800000 31.64851546496383
121900000 20.967911227762855
122000000 37.142031850937975
122100000 36.47703153730172
122200000 35.48933200658596
122300000 37.10865490811671
122400000 37.6190161111625
122500000 41.19818063478589
122600000 40.22753436277519
122700000 40.44915989925402
122800000 39.0116

...

522600000 4.2682488184537535
522700000 4.4358055756725605
522800000 2.272746113050292
522900000 3.266213600884977
523000000 4.879214284582025
523100000 4.988499279405065
523200000 4.689045504867212
523300000 5.991264440296666
523400000 6.829551617598337
523500000 6.260360871453898
523600000 5.1337307274180155
523700000 4.835306690283098
523800000 4.984177376446012
523900000 3.8534206659553636
524000000 2.556677123390623
524100000 3.1438612767383645
524200000 5.186046868139785
524300000 4.918368921810609
524400000 4.57376063348688
524500000 4.783696492593637
524600000 5.435603049660148
524700000 4.752975519899388
524800000 4.540072203472515
524900000 4.4075509852872985
525000000 5.384902732716069
525100000 3.7395411257631666
525200000 2.6247247099544766
525300000 3.1052708664638544
525400000 4.566677771814328
525500000 4.918970310119912
525600000 4.52880336967227
525700000 4.632325612070361
525800000 5.156449114978846
525900000 5.131544310253485
526000000 4.638628169319723
526100000 4.6

551500000 3.6454899634923947
551600000 3.116192601886948
551700000 2.9925707685281324
551800000 4.507527666155123
551900000 4.178130735678408
552000000 4.161685704861643
552100000 3.896928059343849
552200000 4.157968828429144
552300000 4.132556756147978
552400000 3.702975823577238
552500000 2.8596648653094667
552600000 3.293483317262164
552700000 3.553922196436376
552800000 3.1775177114440383
552900000 2.397603481625396
553000000 3.5401967299002886
553100000 3.6292332026608642
553200000 2.808482421110502
553300000 3.8908651009780324
553400000 4.064051081331439
553500000 3.958787044933135
553600000 3.630938058093966
553700000 2.517989589452227
553800000 3.57738584382594
553900000 3.534385069163336
554000000 3.1162790718214395
554100000 2.253590431304138
554200000 3.4924199077043894
554300000 3.5027283898815513
554400000 2.8486638467977445
554500000 3.796659867604713
554600000 3.9232009880850938
554700000 3.8305954193822433
554800000 3.7526368592131685
554900000 2.310602049750924
5550000

579000000 2.3441780766354485
579100000 1.7671022037697435
579200000 2.0232615382288635
579300000 2.1585591375721846
579400000 2.202786378349356
579500000 2.116096669254746
579600000 2.145005635583459
579700000 1.789701146385853
579800000 1.9334568511355936
579900000 1.9891381195038782
580000000 2.2786527826780065
580100000 2.365771747342467
580200000 1.944512934454326
580300000 1.7876256823315282
580400000 1.9002052645779652
580500000 1.9265469517220033
580600000 2.4464430284266485
580700000 2.5034799780708172
580800000 1.7505865692455205
580900000 1.8248499567167797
581000000 1.8600480760878266
581100000 1.770265384709035
581200000 2.508580445428335
581300000 2.544032621380498
581400000 1.7035583746970817
581500000 1.8019866795962665
581600000 1.81777103766646
581700000 1.7201171848053416
581800000 2.471409902771865
581900000 2.4501445111632405
582000000 1.6516198066599361
582100000 1.839719821720928
582200000 1.8313234833251992
582300000 1.7064313893357688
582400000 2.416734419892829

607500000 1.1653076547303511
607600000 0.9340501603487419
607700000 1.2458563399191398
607800000 0.9382458458301696
607900000 1.1872288770268273
608000000 0.738458704797566
608100000 1.1175675813508152
608200000 0.9722886039643381
608300000 1.1436993145180088
608400000 0.9144691703586525
608500000 1.1688261201029924
608600000 0.7217356551552793
608700000 1.1356660039143873
608800000 1.060130484068332
608900000 1.0413594762987792
609000000 0.9026301300772243
609100000 1.1442804112472666
609200000 0.7119974333748499
609300000 1.1240814519426379
609400000 1.1022201512108611
609500000 0.9201694661416769
609600000 0.8759783378534384
609700000 1.1204184297796687
609800000 0.6793378664317369
609900000 1.1025845206106906
610000000 1.1132142798819276
610100000 0.7764012027106245
610200000 0.8837216956759824
610300000 1.088828840515942
610400000 0.6785336605831245
610500000 1.0782583250506474
610600000 1.0847197558857309
610700000 0.7248105206977963
610800000 0.9100175599397288
610900000 1.08496

best so far: 0
type: [2, 1, 1, 67, 1] 134
cases of this type: 300763
633900000 0.005441682285646596
634000000 0.0009956200488209725
634100000 0.0003340591867566108
best so far: 0
type: [2, 1, 67, 1, 1] 134
cases of this type: 4489
best so far: 0
type: [2, 67, 1, 1, 1] 134
cases of this type: 67
best so far: 0
type: [67, 1, 1, 1, 2] 134
cases of this type: 16
best so far: 0
type: [67, 1, 1, 2, 1] 134
cases of this type: 8
best so far: 0
type: [67, 1, 2, 1, 1] 134
cases of this type: 4
best so far: 0
type: [67, 2, 1, 1, 1] 134
cases of this type: 2
best so far: 0
type: [134, 1, 1, 1, 1] 134
cases of this type: 1
best so far: 0
all cases: 634149671
best: 0
